In [1]:
import sys
sys.path.append('/mnt/d/ariel2/code/core/')
sys.path.append('d:/ariel2/code/core/')
sys.path.append('/kaggle/input/my-ariel2-library')
import kaggle_support as kgs
import ariel_model
import ariel_gp
import time
import numpy as np
import multiprocess
from tqdm import tqdm

local


In [2]:
train_data = kgs.load_all_train_data()
kgs.debugging_mode = 1
model = ariel_gp.PredictionModel()
model.run_in_parallel = True
model.model_options.use_training_labels = True
#model.model_options.make_noise_consistent = True
model.model_options.n_samples_sigma_est = 2
kgs.sanity_checks_active = False
model.train(train_data)

In [5]:
model.run_in_parallel = False
def f(d):
    import kaggle_support as kgs
    import numpy as np
    kgs.debugging_mode = 1
    kgs.sanity_checks_active = False
    model = kgs.dill_load(kgs.temp_dir + '/parallelx.pickle')
    d = model.infer([d])[0]
    return [model.results['model_mean'].get_parameters(), model.results['model_mean'].m['signal'].m['main'].m['transit'].transit_params]
data = train_data
kgs.dill_save(kgs.temp_dir + '/parallelx.pickle', model)
with multiprocess.Pool(6) as p:                   
    result = list(tqdm(
        p.imap(f, data),
        total=len(data),
        desc="Processing in parallel", smoothing = 0.05
        ))
#kgs.dill_save(kgs.temp_dir + '/prep3.pickle', result)

Processing in parallel: 100%|█████████████████████████████████████████████████████████████| 1/1 [00:32<00:00, 32.30s/it]

local
SpawnPoolWorker-7 7
CUDA_VISIBLE_DEVICES= 0
New solver/alternative transit parameters fallback for planet id 1547333761
Attempting chop
New solver/alternative transit parameters fallback for planet id 1547333761
Chop used on planet id 1547333761


In [6]:
[r[0].shape[0] for r in result]

[276609]